In [25]:
import pandas as pd
from collections import Counter

i_m = 0.053098482833127306
e = 0.005

In [26]:
def gcd(a, b):
    r_0, r_1 = a, b 
    u_0, u_1 = 1, 0
    v_0, v_1 = 0, 1

    while r_1 != 0:
        q = r_0 // r_1
        r_0, r_1 = r_1, r_0 - q*r_1
        u_0, u_1 = u_1, u_0 - q*u_1
        v_0, v_1 = v_1, v_0 - q*v_1

    return r_0, u_0, v_0

def inv(a, b):
    d, u, v = gcd(a, b)
    if d != 1:
        return "No inverse exists"
    else:
        return u

def mod_comp(a, b, n):
    d = gcd(a, n)[0]
    s = []

    if d == 1:
        return (inv(a, n) * b) % n, 1
    elif b % d != 0: 
        return "No solutions.", 0
    else:
        a = a / d
        b = b / d
        n = n / d
        x_0 = (inv(a, n) * b) % n

        for i in range(0, d):
            s.append(x_0 + i * n)

        return s, d


In [27]:
alp = ['а','б','в','г','д','е','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ы','ь','э','ю','я']

m = len(alp)

fr_bigrams_lng = ['ст', 'но', 'то', 'на', 'ен']
fr_sym_lng = ['о', 'е', 'а', 'и', 'т', 'н', 'с', 'р', 'в', 'л']
rar_sym_lng = ['ф', 'э', 'щ', 'ц', 'ш', 'ю', 'ж', 'х', 'й', 'ч']
fr_trigrams_lng = ['сто', 'ено', 'нов', 'тов', 'ово', 'ова']

name_f = 'V7 (1).txt'

def process_text(name):
    f = open(name_f, 'r', encoding='utf-8')
    text = f.read()
    f.close()
    text = text.lower()
    text = text.replace('ъ', 'ь')
    text = text.replace('ё', 'е')
    text_n = ""
    for l in text:
        if l in alp:
            text_n += l
        else:
           text_n += ' '
    text_n =  " ".join(text_n.split())
    return text_n

text = process_text(name_f)

def bigrams_without_sp(text):
    count = {}
    t = 1
    while t < len(text):
        p = text[t - 1]
        l = text[t]
        if (p + l) not in count:
            count[p + l] = 1
        if (p + l) in count:  
            count[p + l] = count[p + l] + 1
        t = t + 2
    return count


In [28]:
def match(y):
    return alp.index(y[0]) * m + alp.index(y[1])

def unmatch(X):
    x1 = X // m
    x2 = X - x1*m
    return x1, x2

def get_keys(text):
    a = sorted(bigrams_without_sp(text).items(), key=lambda x:x[1], reverse = True)
    fr_bigrams_txt = []
    for i in range(0, 5):
        fr_bigrams_txt.append(a[i][0])

    keys = []
    for i in range(len(fr_bigrams_txt)):
        for k in range(len(fr_bigrams_lng)):
            for j in range(len(fr_bigrams_txt)):
                    for l in range(len(fr_bigrams_lng)):
                        if i!=j:
                            if k!=l:
                                x1 = match(fr_bigrams_txt[i])
                                x2 = match(fr_bigrams_txt[j])
                                y1 = match(fr_bigrams_lng[k])
                                y2 = match(fr_bigrams_lng[l])
                                y = (y1 - y2) % (m **2)
                                x = (x1 - x2) % (m **2)
                                a1, t = mod_comp(y, x, m **2)
                                if t == 1:
                                    b1 = (y - a1 * x) % (m **2)
                                elif t == 0:
                                    continue
                                else:
                                    b1 = []
                                    b1.append((y - i * x) % (m **2) for i in a1)
                                keys.append((a1, b1))
    return keys

In [29]:
def decypher(k, text):
    text_n = ""
    Y = list(bigrams_without_sp(text).keys())
    X = {}
    for i in range(0, len(Y)):
        X[i] = inv(k[0], m**2) * (match(Y[i]) - k[1]) 
        X[i] = X[i] % (m **2)

    t = 1
    while t < len(text):
        p = text[t - 1]
        l = text[t]
        ind = Y.index(p + l)
        x1, x2 = unmatch(X[ind])
        text_n += alp[x1] + alp[x2]
        t = t + 2
    return text_n

def affi(text):
    i = 0
    for j in Counter(text).values():
        i += j * (j - 1) 
    i = i / (len(text) * (len(text) - 1))
    return i

def check_affi(t):
    if abs(affi(t) - i_m) < e:
        return 1
    return 0

def check_fr_sym(t):
    tsort_sym = sorted(Counter(t).items(), key=lambda x:x[1], reverse = True)
    fr_sym_txt = []
    for i in range(0, 5):
        fr_sym_txt.append(tsort_sym[i][0])
    i = 0
    for j in fr_sym_txt:
        if j in fr_sym_lng:
            i = i + 1
    if i > 5:
        return 1
    return 1

def check_rar_sym(t):
    tsort_sym = sorted(Counter(t).items(), key=lambda x:x[1], reverse = True)
    fr_sym_txt = []
    for i in range(0, 5):
        fr_sym_txt.append(tsort_sym[i][0])
    i = 0
    for j in fr_sym_txt:
        if j in rar_sym_lng:
            i = i + 1
    if i > 5:
        return 1
    return 1

def check_fr_bigrams(t):
    tsort_big = sorted(bigrams_without_sp(t).items(), key=lambda x:x[1], reverse = True)
    fr_big_txt = []
    for i in range(0, 5):
        fr_big_txt.append(tsort_big[i][0])
    i = 0
    for j in fr_big_txt:
        if t in fr_sym_lng:
            i = i + 1
    if i > 2:
        return 1
    return 1

def checking_the_mf(keys, text):
    true = []
    for i in keys:
        t = decypher(i, text)
        if check_affi(t) == 1:
            if check_fr_sym(t) == 1:
                if check_rar_sym(t) == 1:
                    if check_fr_bigrams(t) == 1:
                        true.append(i)
        continue
    return true

In [30]:
v = get_keys(text)
print(checking_the_mf(v, text))

[(926, 932), (35, 822)]
